In [2]:
import networkx as nx
from urllib.parse import urlparse
import numpy as np
from operator import itemgetter
from bson import json_util, ObjectId

In [3]:
with open('/home/elisa/Documents/Projetos/TCC/data/articles_data/charliehebdo.json') as f:
    a = f.read()
    list_of_dict = json_util.loads(a)
    
del_pos = []
for pos,i in enumerate(list_of_dict):
    if 'published' not in i:
        i['published'] = i['updated']
    if i['published'].year < 2015:
        del_pos.append(i)
        
for i in del_pos:
    list_of_dict.remove(i)

data = sorted(list_of_dict, key=itemgetter('published'))
del data[0]

In [4]:
dists_triu = np.loadtxt('/home/elisa/Documents/Projetos/TCC/data/charlie/matrix_cosine_similarity.csv', delimiter=',')

In [5]:
def get_pos(data, pub_i, column_list, time_max, sim_min,outs):
    sim = max(column_list)
    if sim < sim_min:
        return None

    pos = column_list.index(sim)
    time_dif = (data[pos]['published'] - pub_i).total_seconds()/3600
    if pos in outs:
        column_list[pos] = 0
        get_pos(data,pub_i, column_list, time_max, sim_min,outs)
    elif time_dif > time_max:
        column_list[pos] = 0
        get_pos(data,pub_i, column_list, time_max, sim_min,outs)
    else:
        return pos

In [6]:
def create_date(pub1,pub2,s):
    dif = (pub2-pub1).total_seconds()/3600
    return round((dif/s))

In [7]:
dates=[]
def create_graphml(dists_triu, data, time_max=164, sim_min=0.8):
    size = dists_triu.shape[0]
    G = nx.DiGraph()
    G.add_node(0, step=0, date=0,domain=urlparse(data[0]['link']).netloc)
    date_init = data[0]['published']
    outs = []
    for i in range(1,size):
        pub_i = data[i]['published']
        column = list(dists_triu[:,i])
        pos = get_pos(data, pub_i, column, time_max, sim_min, outs)

        if pos != None:
            if pos not in G.nodes():
                domain_1 = urlparse(data[pos]['link']).netloc
                date_1 = create_date(date_init,data[pos]['published'], 5)
                dates.append(date_1)
                G.add_node(pos, date=date_1, domain=domain_1)
            if i not in G.nodes():
                domain_2 = urlparse(data[i]['link']).netloc
                date_2 = create_date(date_init, pub_i, 5)
                dates.append(date_2)
                G.add_node(i, date=date_2, domain=domain_2)
    
            G.add_edge(pos,i)
        else:
            outs.append(i)
    return G

In [8]:
def cria_atribs_step(G,list):
    for i in list:
        for node in G.successors(i):
            G.node[node]['step'] = G.node[i]['step'] + 15
        cria_atribs_step(G,G.successors(i))
    return G

In [9]:
G = create_graphml(dists_triu, data)
G = cria_atribs_step(G,[0])

In [10]:
nx.write_graphml(G, 'gephi/grafo_original.graphml')

In [11]:
plt.hist(dates,bins=40)
plt.show()

NameError: name 'plt' is not defined

##Grafo simulado

In [37]:
H = np.loadtxt(open('/home/elisa/Documents/Projetos/TCC/data/charlie/graph_complete.csv'), delimiter=",")
Infects = np.loadtxt(open('/home/elisa/Documents/Projetos/TCC/data/charlie/Infects.csv'), delimiter=",")
original_graph = nx.read_gpickle('/home/elisa/Documents/Projetos/TCC/data/charlie/original_graph.gpickle')

In [38]:
H.shape

(1786, 1786)

In [21]:
with open('/home/elisa/Documents/Projetos/TCC/data/charlie/original_graph_domains_each_node.txt') as f:
    domains = f.read().splitlines()

In [22]:
domains

['www.ptjornal.com',
 'www.meioemensagem.com.br',
 'br.noticias.yahoo.com',
 'estadao.feedsportal.com',
 'estadao.feedsportal.com',
 'estadao.feedsportal.com',
 'estadao.feedsportal.com',
 'estadao.feedsportal.com',
 'br.noticias.yahoo.com',
 'www.dgabc.com.br',
 'www.jb.com.br',
 'opiniaoenoticiascombr.feedsportal.com',
 'br.noticias.yahoo.com',
 'feeds.jn.pt',
 'www.gazetadopovo.com.br',
 'br.noticias.yahoo.com',
 'estadao.feedsportal.com',
 'feeds.jn.pt',
 'bahianoticias.feedsportal.com',
 'www.infomoney.com.br',
 'feeds.jn.pt',
 'g1.globo.com',
 'estadao.feedsportal.com',
 'www.brasilimprensalivre.com.br',
 'estadao.feedsportal.com',
 'feeds.jn.pt',
 'g1.globo.com',
 'www.jb.com.br',
 'feeds.jn.pt',
 'www.jb.com.br',
 'www.jb.com.br',
 'estadao.feedsportal.com',
 'br.noticias.yahoo.com',
 'estadao.feedsportal.com',
 'aquiacontece.com.br',
 'feeds.jn.pt',
 'www.ptjornal.com',
 'estadao.feedsportal.com',
 'www.gazetadopovo.com.br',
 'g1.globo.com',
 'www.oab.org.br',
 'g1.globo.com',

In [39]:
original_graph.node[494]

{'_id': ObjectId('54afdd9bdcccdd1dd8d80fb4'),
 'children': [],
 'date': datetime.datetime(2015, 1, 9, 13, 27, 57, tzinfo=<bson.tz_util.FixedOffset object at 0x7fe4c4ed3d30>),
 'domain': 'br.noticias.yahoo.com'}

In [40]:
Infects.shape

(3001, 1786)

In [41]:
def graph_simu_nx(Infects, G, domains):
    s = Infects.shape[1]
    graph_sim = nx.DiGraph()

    #I_previous é o vetor de indices dos nos infectados no tempo 0
    I_previous = np.where(Infects[0]==1)[0]
    graph_sim.add_node(I_previous[0], time=0)

    #done é a lista com os indices dos nos que ja descobrimos o pai
    done = [I_previous[0]]

    for passo, status in enumerate(Infects[1:]):
        t_previous = passo
        t_current = passo+1
        
        I = list(np.where(status==1)[0])
        I_current = I.copy()
        for i in done:
            if i in I_current:
                I_current.remove(i)

        if len(I_current) == 0:
            continue

        #se temos mais de uma possibilidade de pai
        if len(I_previous) == 1:
            for node in I_current:
                if node not in graph_sim.nodes():
                    graph_sim.add_node(node,time=t_current, domain=domains[node])
                if I_previous[0] not in graph_sim.nodes():
                    graph_sim.add_node(I_previous[0],time=t_previous,domain=domains[I_previous[0]])
                graph_sim.add_edge(I_previous[0],node)
                done.append(node)

        else:
            #calcular pai para cada no que foi infectado
            for node in I_current:
                #pegando a probabilidade de cada no ja infectado ter infectado o no node
                influences = G[node]
                probs = np.zeros(s)

                for i in I_previous:
                    probs[i] = influences[i]
                if sum(probs) != 0 :
                    probs = probs/sum(probs)
                pos = np.random.choice(range(s), p=probs, size=1)
                
                if node not in graph_sim.nodes():
                    graph_sim.add_node(node,time=t_current,domain=domains[node])
                if pos[0] not in graph_sim.nodes():
                    graph_sim.add_node(pos[0],time=t_previous,domain=domains[node])
                    
                graph_sim.add_edge(pos[0], node)
                done.append(node)
                if pos == 0:
                    print(probs)
        I_previous = I

    return graph_sim

In [42]:
graph_simulated = graph_simu_nx(Infects, H, domains)

In [44]:
graph_simulated.node[0]['step']=0

In [45]:
G.node[0]

{'date': 0, 'domain': 'www.ptjornal.com', 'step': 0}

In [34]:
a =[1,2,3,4,5]

In [36]:
a[:5]

[1, 2, 3, 4, 5]

In [67]:
graph_simulated.node[0]

{'step': 0}

In [46]:
graph_simulated = cria_atribs_step(graph_simulated,[0])

In [47]:
nx.write_graphml(graph_simulated, '/home/elisa/Documents/Projetos/TCC/Notebook/gephi/graph_simulated_nx.graphml')